In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline 
import random
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llms-you-cant-please-them-all/sample_submission.csv
/kaggle/input/llms-you-cant-please-them-all/test.csv
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/LICENSE
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/README.md
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/tokenizer.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/tokenizer_config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/model.safetensors
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/.gitattributes
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/generation_config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2/figures/benchmark.jpg


In [5]:
# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

# Load test dataset
test_df = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')
submission_df = pd.DataFrame(columns=["id", "essay"])
submission_df["id"] = test_df["id"]

Using device: GPU


In [7]:
## Load offline model from Kaggle dataset 
model_path = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2"
generator = pipeline("text-generation", model=model_path, device=device)

Device set to use cuda:0


In [8]:
# Define Essay Generation Function to generate diverse essays
def generate_diverse_essay(topic):
    """Generates an essay using diverse writing styles to maximize LLM disagreement."""
    prompts = [
        f"Write a formal academic essay on {topic}, including citations and structured arguments.",
        f"Write a satirical critique on {topic}, exaggerating contradictions and ironies.",
        f"Write a philosophical dialogue where two characters debate {topic} with opposing views.",
        f"Write a poem that metaphorically explores {topic} using vivid imagery.",
        f"Write a controversial argument about {topic}, making bold claims with limited evidence.",
    ]

    selected_prompt = random.choice(prompts)  # Randomly pick a style
    essay = generator(selected_prompt, max_length=150, truncation=True)[0]['generated_text']
    return essay

In [9]:
# Generate essays
test_df['essay'] = test_df['topic'].apply(generate_diverse_essay)
submission_df['essay'] = test_df['essay']

In [10]:
print (submission_df['essay'].values)

['Write a satirical critique on Compare and contrast the importance of self-reliance and adaptability in healthcare., exaggerating contradictions and ironies. Include a reference to a specific disease or condition that illustrates this critique.\nOkay, so I need to write a satirical critique comparing the importance of self-reliance and adaptability in healthcare. The user wants me to exaggerate the contradictions and ironies, and include a specific disease or condition. Let me think about how to approach this.\n\nFirst, I should understand the core concepts: self-reliance and adaptability in healthcare. Self-reliance means patients or individuals taking on their own medical care, relying on their own resources. Adaptability refers to the ability to change plans or respond'
 'Write a philosophical dialogue where two characters debate Evaluate the effectiveness of management consulting in addressing conflicts within marketing. with opposing views. The two characters should be the author

In [11]:
# Save submission file
submission_df[["id", "essay"]].to_csv("submission.csv", index=False)

print("Submission file created successfully!")

Submission file created successfully!
